**Задание 3: Сравнение производительности CPU vs CUDA (20 баллов)** \
**3.1 Подготовка данных (5 баллов)**

In [1]:
import torch
tensor1 = torch.rand(64, 1024, 1024)# Матрица размером 64 × 1024 × 1024, заполненная случайными числами от 0 до 1
tensor2 = torch.rand(128, 512, 512)
tensor3 = torch.rand(256, 256, 256)

**3.2 Функция измерения времени (5 баллов)**

In [19]:
import time

def time_cpu(func, *args):
    """
    Измеряет время выполнения операции на CPU
    """
    start = time.time() #начала выполнения функции
    func(*args)  #вызываем функцию
    end = time.time() #время окончания
    return end - start
def time_gpu(func, *args):
    """
    Измеряет время выполнения операции на GPU 
    """
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    
    start.record() #начала выполнения функции
    func(*args) #вызываем функцию
    end.record() #фиксируем момент сразу ПОСЛЕ вызова операции
    
    torch.cuda.synchronize()  # ждём завершения всех операций на GPU
    return start.elapsed_time(end) / 1000  # возвращаем время в секундах

**3.3 Сравнение операций (10 баллов)**

In [26]:
tensor1 = torch.rand(64, 1024, 1024)#поменял размеры тензоров, потому что невозможно умножение двух матриц, где столбец первой не соответствует строке второй и тд
tensor2 = torch.rand(64, 1024, 1024)    
tensor3 = torch.rand(64, 1024, 1024)    
def matmul_op(a, b):
    """
    Матричное умножение
    """
    return torch.matmul(a,b)

def add_op(a, b):
    """
    Поэлементное сложение
    """
    return a + b

def mul_op(a, b):
    """
    Поэлементное умножение
    """
    return a * b

def transpose_op(a):
    """
    Транспонирование
    """
    return a.T

def sum_op(a):
    """
    Вычисление суммы всех элементов
    """
    return a.sum()

device = 'CPU'

results_CPU = {
    "Мат. умножение": time_cpu(matmul_op, tensor1, tensor2),
    "Сложение": time_cpu(add_op, tensor2, tensor3),
    "Поэлементное умножение": time_cpu(mul_op, tensor2, tensor3),
    "Транспонирование": time_cpu(transpose_op, tensor1),
    "Сумма элементов": time_cpu(sum_op, tensor3),
}

device = 'cuda'
t1 = tensor1.to(device)
t2 = tensor2.to(device)
t3 = tensor3.to(device)

results_GPU = {
    "Мат. умножение": time_gpu(matmul_op, t1, t2),
    "Сложение": time_gpu(add_op, t2, t3),
    "Поэлементное умножение": time_gpu(mul_op, t2, t3),
    "Транспонирование": time_gpu(transpose_op, t1),
    "Сумма элементов": time_gpu(sum_op, t3),
}
print(f"{'Операция':<25} | {'CPU (мс)':>10} | {'GPU (мс)':>10} | {'Ускорение':>10}")
print("-" * 60)
for key in results_CPU:
    cpu_time_ms = results_CPU[key] * 1000
    gpu_time_ms = results_GPU[key] * 1000
    speedup = cpu_time_ms / gpu_time_ms
    print(f"{key:<25} | {cpu_time_ms:10.2f} | {gpu_time_ms:10.2f} | {speedup:10.2f}x")



Операция                  |   CPU (мс) |   GPU (мс) |  Ускорение
------------------------------------------------------------
Мат. умножение            |     698.88 |      92.24 |       7.58x
Сложение                  |      58.08 |       5.92 |       9.81x
Поэлементное умножение    |      40.95 |       5.89 |       6.95x
Транспонирование          |       0.04 |       0.00 |      19.21x
Сумма элементов           |       9.89 |       1.54 |       6.42x
